# Importing packages

In [1]:
import os
import sys
import fileinput
import subprocess
import re

# Checking the working directory

In [2]:
os.getcwd()

'C:\\Users\\detemplj\\Downloads'

# Changing fertiliser amount in APSIM script

Took this code from this Stack Overflow: https://stackoverflow.com/questions/17140886/how-to-search-and-replace-text-in-a-file.
Prep work for this included replacing the fertilizer value with "<fertiliser>" (to make it easy to find in the file)

In [3]:
# Read in the file
with open('maize_py_example.apsimx', 'r') as file :
  filedata = file.read()

# Replace the target string
filedata = filedata.replace('<fertiliser>', '10000000')

# Write the file out again
with open('maize_py_example_edited.apsimx', 'w') as file:
  file.write(filedata)

# Running the APSIM script in shell

Using the package "subprocess" (imported above). Sourced from ChatGPT

In [4]:
list_files = subprocess.check_output(["apsim", "run", "maize_py_example_edited.apsimx"], shell=True)
list_files = list_files.decode("utf-8")
#print("The output was:")
print(list_files)

# Extracting .db contents with SQLite

Sourced from ChatGPT

In [5]:
%load_ext sql
%sql sqlite:///c:/Users/detemplj/Downloads/maize_py_example_edited.db

Getting the column names

In [6]:
%%sql
SELECT *
FROM PRAGMA_TABLE_INFO('Report')

 * sqlite:///c:/Users/detemplj/Downloads/maize_py_example_edited.db
Done.


cid,name,type,notnull,dflt_value,pk
0,CheckpointID,INTEGER,0,None,0
1,SimulationID,INTEGER,0,None,0
2,Zone,TEXT,0,None,0
3,Clock.Today,date,0,None,0
4,Maize.Phenology.CurrentStageName,TEXT,0,None,0
5,Maize.AboveGround.Wt,REAL,0,None,0
6,Maize.AboveGround.N,REAL,0,None,0
7,Maize.Grain.Wt,REAL,0,None,0
8,Maize.Grain.Size,REAL,0,None,0
9,Maize.Grain.NumberFunction,REAL,0,None,0


Getting all information of the record with the highest AboveGround.Wt

In [7]:
%%sql
SELECT * FROM Report
WHERE "Maize.AboveGround.Wt" = (SELECT MAX("Maize.AboveGround.Wt") FROM Report);

 * sqlite:///c:/Users/detemplj/Downloads/maize_py_example_edited.db
Done.


CheckpointID,SimulationID,Zone,Clock.Today,Maize.Phenology.CurrentStageName,Maize.AboveGround.Wt,Maize.AboveGround.N,Maize.Grain.Wt,Maize.Grain.Size,Maize.Grain.NumberFunction,Maize.Grain.N,Maize.Total.Wt,Maize.Organs.Length
1,1,Field,1995-12-31 12:00:00,,666.4856588928575,6.771708666760553,34.387912974820644,0.010890954176908168,3157.4747644914823,1.7193956487410325,749.1247747249881,6


Getting just the Maize.AboveGround.Wt variable

In [8]:
%%sql
SELECT "Maize.AboveGround.Wt" FROM Report
WHERE "Maize.AboveGround.Wt" = (SELECT MAX("Maize.AboveGround.Wt") FROM Report);

 * sqlite:///c:/Users/detemplj/Downloads/maize_py_example_edited.db
Done.


Maize.AboveGround.Wt
666.4856588928575


Uses subprocess, re packages (imported above)
The max number of the variable (Maize.AboveGround.Wt) is stored in "value" variable. Sourced from ChatGPT

In [9]:
output = subprocess.check_output(["sqlite3", "c:/Users/detemplj/Downloads/maize_py_example_edited.db", "-cmd", 'SELECT * FROM Report WHERE "Maize.AboveGround.Wt" = (SELECT MAX("Maize.AboveGround.Wt") FROM Report);'], shell=True)
output = output.decode("utf-8")

match = re.search(r'\d+\.\d+', output)
if match:
    value = match.group()
    print("The value is:", value)
else:
    print("No value found.")


The value is: 666.485658892858
